## Matapath2vec

Metapath2vec [[1]](#fn1) extends the idea of DeepWalk and node2vec to heterogeneous networks. It introduces meta-path based random walks to generate heterogeneous neighborhoods. In this way, it can outpeform algorithms which ignore heterogeneous nature of graphs, such as node2vec. Namely, the heterogeneous random walks are biased to highly visible types of nodes and concetrated nodes. The metapath2vec algorithm uses meta-path-based random walk strategy to ensure that the semantic relationships can be incorporated into the skip-gram model.

As an example of a heterogeneous network we use a citation network from the ArnetMiner project [[2]](#fn2) which provides a selection of large citation networks compiled from computer science bibliography sources such as DBLP and ACM. We use the DBLP citation network version 10 which is conveniently stored in the JSON format.

The following packages are required to run this notebook:

- stellargraph==1.2.1
- networkx==2.5
- remotezip-0.9.2
- pyx==0.15

-----
<span id="fn1"> [1] Yuxiao Dong, Nitesh V. Chawla, and Ananthram Swami. 2017. Metapath2vec: Scalable Representation Learning for Heterogeneous Networks. In Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD '17). Association for Computing Machinery, New York, NY, USA, 135–144. </span>

<span id="fn2"> [2] Jie Tang, Jing Zhang, Limin Yao, Juanzi Li, Li Zhang, and Zhong Su. ArnetMiner: Extraction and Mining of Academic Social Networks. In Proceedings of the Fourteenth ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (SIGKDD'2008). pp.990-998. </span>

-----

The complete [DBLP-Citation-network V10](https://www.aminer.cn/citation) is a 1.7GB zip file which contains description of a graph with more than 3M nodes and 25M edges. We use only a small part of this network stored in the file `dblp-ref/dblp-ref-3.json`.

In [1]:
from remotezip import RemoteZip
import json
import os

if not os.path.exists('data/dblp-ref-3.json'):
    with RemoteZip('https://lfs.aminer.cn/lab-datasets/citation/dblp.v10.zip') as zip:
        with zip.open('dblp-ref/dblp-ref-3.json') as dblp:
            with open('data/dblp-ref-3.json', 'w') as ofp:
                ofp.write(dblp.read().decode('utf-8'))
          
with open('data/dblp-ref-3.json') as fp:
    lines = [json.loads(line) for line in fp if line]
print(len(lines))

ModuleNotFoundError: No module named 'remotezip'

Every line in the file is a JSON object containing some or all of the following attributes: id, title, authors, venue, year, n_citation, references, and abstract. We will build a network accoding to the following schema.

In [2]:
from pyx import *
h = 0.707
c = canvas.canvas()
c.stroke(path.circle(4, 4, 1))
c.stroke(path.circle(8, 0, 1))
c.stroke(path.circle(0, 0, 1))
c.stroke(path.line(h, h, 4-h, 4-h), [deco.earrow(size=0.5), deco.curvedtext("authorOf")])
c.stroke(path.line(4+h, 4-h, 8-h, h), [deco.earrow(size=0.5), deco.curvedtext("publishedIn")])
c.stroke(path.curve(3, 4, 1, 5, 3, 7, 4,5), [deco.earrow(size=0.5), deco.curvedtext("cites")])
c.text(-0.5, 0, 'author')
c.text(3.6, 4, 'paper')
c.text(7.5, 0, 'venue')
c

ModuleNotFoundError: No module named 'pyx'

We transform the input data into a network. First we create an empty directed network and populate it with nodes and edges according to the input data. We consider only complete data, i.e., the venue, authors and references must be present. We ignore references to papers not present in our subset.

In [3]:
import networkx as nx
g = nx.DiGraph()
paper_ids = set()
venues = set()
authors = set()
for line in lines:
    if 'references' in line and line['venue'] and line['authors']:
        paper_ids.add(line['id'])
for line in lines:
    if 'references' in line and line['venue'] and line['authors']:
        if line['id'] in paper_ids:
            g.add_node(line['id'], label='paper')
            for author in line['authors']:
                g.add_node(author, label='author')
                g.add_edge(author, line['id'], label='authorOf') 
                authors.add(author)

            g.add_node(line['venue'], label='venue')
            g.add_edge(line['id'], line['venue'], label='publishedIn')
            venues.add(line['venue'])

            for cited in line['references']:
                if cited in paper_ids:
                    g.add_node(cited, label='paper')
                    g.add_edge(line['id'], cited, label='cites')
print(g.number_of_nodes(), g.number_of_edges())

NameError: name 'lines' is not defined

We decide to ignore all weakly connected components except the largest one.

In [4]:
from collections import Counter
from pprint import pprint
cc_sizes = Counter()
for cc in nx.weakly_connected_components(g):
    cc_sizes[len(cc)] += 1
pprint(sorted(cc_sizes.items()))
max_cc = max(nx.weakly_connected_components(g), key=len)    
g = g.subgraph(max_cc).copy()
print(f'V={g.number_of_nodes()}, E={g.number_of_edges()}, density: {nx.density(g):.6f}')

[]


ValueError: max() arg is an empty sequence

In order to run metapath2vec, we first load the graph into the `StellarGraph` data structure.

In [5]:
import stellargraph
sg = stellargraph.StellarGraph.from_networkx(g)
print(sg.info())

ModuleNotFoundError: No module named 'stellargraph'

To run metapath2vec, we set metapath schemas which define types of nodes the random walker is allowed to transition from its current location. For example, a meta path $P \rightarrow P$ represents a citation, while $A \rightarrow P \rightarrow V \rightarrow P \rightarrow A$ represents authors publishing in the same venue. Because the graph is large, we will configure the random walker to do only 3 walks of length 10 per node.

In [6]:
metapaths = [
    ['author', 'paper', 'author'], # co-authorship
    ['paper', 'venue', 'paper'],   # papers at the same venue
    ['author', 'paper','venue','paper','author'] # authors at the same venue     
]

walker = stellargraph.data.UniformRandomMetaPathWalk(sg,
                                                     n=3,        # walks per root node
                                                     length=10,   # walk length
                                                     metapaths=metapaths,
                                                     seed=42)
random_walks = walker.run(nodes=list(sg.nodes()))

NameError: name 'stellargraph' is not defined

We display three of the crawled random walks. They follow these meta paths: 
- $A \rightarrow P \rightarrow A \rightarrow P \rightarrow  A \rightarrow  P \rightarrow A \rightarrow  P \rightarrow A \rightarrow  P$ 
- $A \rightarrow P \rightarrow V \rightarrow P \rightarrow A \rightarrow P \rightarrow V \rightarrow P \rightarrow A \rightarrow P$
- $P \rightarrow V \rightarrow P \rightarrow V \rightarrow P \rightarrow V \rightarrow P \rightarrow V \rightarrow P \rightarrow V$

We can recognize the extended patterns of the meta-paths passed to the random walker.

In [7]:
pprint(random_walks[0])
pprint(random_walks[100])
pprint(random_walks[-1])

NameError: name 'random_walks' is not defined

As we have random walks available, we can use word2vec to compute the actual graph embedding. Gensim's implementation of word2vec runs on sentences (lists of tokens). In our case, these are random walks (lists of nodes). 

In [8]:
import gensim
w2v = gensim.models.word2vec.Word2Vec(random_walks, size=100, window=5, workers=3)

NameError: name 'random_walks' is not defined

We computed 100-dimensional embeddings of all nodes while taking into account the heterogeneous nature of the graph. The computed embeddings can be used in a variety of downstream tasks such as node classification, clustering, link prediction, etc. We first visualize the embedding vectors of papers. The 2D projection of paper nodes reveals several tight groups of small sizes and a few dispersed points suggesting that there are only a few outliers.

In [9]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

embeddings = np.array([w2v.wv.get_vector(node) for node in w2v.wv.vocab if node in paper_ids])
print(embeddings.shape)
embeddings_2D = TSNE(n_components=2).fit_transform(embeddings)
x = embeddings_2D[:,0]
y = embeddings_2D[:,1]
figure = plt.figure(figsize=(8, 8))
_ = plt.scatter(x, y, s=0.5, alpha=0.4)
_ = plt.axis('off')

NameError: name 'w2v' is not defined

For comparison, we run the node2vec algorithm, using the same parameter settings and visualize node2vec embeddings of papers.

In [10]:
import node2vec
import random
random.seed(a=42)
n2v = node2vec.Node2Vec(g, dimensions=100, num_walks=3, walk_length=10, workers=4)
n2v_model = n2v.fit(window=5, min_count=5)

ModuleNotFoundError: No module named 'node2vec'

In [11]:
n2v_embeddings = np.array([n2v_model.wv.get_vector(node) for node in n2v_model.wv.vocab if node in paper_ids])
n2v_embeddings_2D = TSNE(n_components=2).fit_transform(n2v_embeddings)
x = n2v_embeddings_2D[:,0]
y = n2v_embeddings_2D[:,1]
figure = plt.figure(figsize=(8, 8))
_ = plt.scatter(x, y, s=0.5, alpha=0.4)
_ = plt.axis('off')

NameError: name 'n2v_model' is not defined

The result is a surprisingly estetic picture which is difficult to compare with the result of metapath2vec above. Nevertheless, we can observe that the number of outliers is much larger in the case of node2vec. Its snake-shaped region in the middle consists of many data points, while the metapath2vec image shows only a few dispersed outliers and a large number of tight, tiny clusters. The actual quality of embeddings in both cases shall be judged based on their performance in downstream tasks.